This notebook is inspired by [Yaroslav Isaienkov's](https://www.kaggle.com/ihelon) great [EDA].(https://www.kaggle.com/ihelon/indoor-location-exploratory-data-analysis)
Please upvote his notebook as well.

# # Indoor Location - Show all trajectories


In this competition, your task is to predict the indoor position of smartphones. **Since the trajectories are collected by a site-surveyor, I have a feeling that the surveyor has a fixed `region` within the building to explore and it is likely that we should always predict the trajectories in that region.**

**In this notebook, we show all the trajectories of a floor plan in training data and check how much area the surveyor covers.**

**<font color='blue'>From the results, it is clear that the surveyor has a well pre-defined regions to cover, which could help us refine where to predict.</font>**

In [ ]:
import os
import glob
import math
import json

from dataclasses import dataclass

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from PIL import Image

In [ ]:
PATH = "../input/indoor-location-navigation"

In [ ]:
def plot_all_floors(path_to_building):
    plt.figure(figsize=(16, 10))
    floor_paths = glob.glob(os.path.join(path_to_building, "*/floor_image.png"))
    for ind, floor_path in enumerate(floor_paths):
        w = math.ceil(len(floor_paths) / 2)
        h = math.ceil(len(floor_paths) / w)
        if len(floor_paths) <= 4:
            w = 4
            h = 1
        plt.subplot(h, w, ind + 1)
        image = cv2.imread(floor_path)  
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.imshow(image)
        plt.axis("off")
        plt.title(floor_path.split("/")[-2], fontsize=16)
    plt.show()

In [ ]:
# copy from https://github.com/location-competition/indoor-location-competition-20/blob/master/io_f.py


@dataclass
class ReadData:
    acce: np.ndarray
    acce_uncali: np.ndarray
    gyro: np.ndarray
    gyro_uncali: np.ndarray
    magn: np.ndarray
    magn_uncali: np.ndarray
    ahrs: np.ndarray
    wifi: np.ndarray
    ibeacon: np.ndarray
    waypoint: np.ndarray


def read_data_file(data_filename):
    acce = []
    acce_uncali = []
    gyro = []
    gyro_uncali = []
    magn = []
    magn_uncali = []
    ahrs = []
    wifi = []
    ibeacon = []
    waypoint = []

    with open(data_filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line_data in lines:
        line_data = line_data.strip()
        if not line_data or line_data[0] == '#':
            continue

        line_data = line_data.split('\t')

        if line_data[1] == 'TYPE_ACCELEROMETER':
            acce.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_ACCELEROMETER_UNCALIBRATED':
            acce_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_GYROSCOPE':
            gyro.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_GYROSCOPE_UNCALIBRATED':
            gyro_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_MAGNETIC_FIELD':
            magn.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_MAGNETIC_FIELD_UNCALIBRATED':
            magn_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_ROTATION_VECTOR':
            ahrs.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_WIFI':
            sys_ts = line_data[0]
            ssid = line_data[2]
            bssid = line_data[3]
            rssi = line_data[4]
            lastseen_ts = line_data[6]
            wifi_data = [sys_ts, ssid, bssid, rssi, lastseen_ts]
            wifi.append(wifi_data)
            continue

        if line_data[1] == 'TYPE_BEACON':
            ts = line_data[0]
            uuid = line_data[2]
            major = line_data[3]
            minor = line_data[4]
            rssi = line_data[6]
            ibeacon_data = [ts, '_'.join([uuid, major, minor]), rssi]
            ibeacon.append(ibeacon_data)
            continue

        if line_data[1] == 'TYPE_WAYPOINT':
            waypoint.append([int(line_data[0]), float(line_data[2]), float(line_data[3])])

    acce = np.array(acce)
    acce_uncali = np.array(acce_uncali)
    gyro = np.array(gyro)
    gyro_uncali = np.array(gyro_uncali)
    magn = np.array(magn)
    magn_uncali = np.array(magn_uncali)
    ahrs = np.array(ahrs)
    wifi = np.array(wifi)
    ibeacon = np.array(ibeacon)
    waypoint = np.array(waypoint)
    
#     print(acce.shape)
#     print(acce_uncali.shape)
#     print(gyro.shape)
#     print(gyro_uncali.shape)
#     print(magn.shape)
#     print(magn_uncali.shape)
#     print(ahrs.shape)
#     print(wifi.shape)
#     print(ibeacon.shape)
#     print(waypoint.shape)
    
    return ReadData(acce, acce_uncali, gyro, gyro_uncali, magn, magn_uncali, ahrs, wifi, ibeacon, waypoint)

In [ ]:
# copy from https://github.com/location-competition/indoor-location-competition-20/blob/master/visualize_f.py


def save_figure_to_html(fig, filename):
    fig.write_html(filename)

def get_lists_of_one_trajectory(trajectory, full_name=True):
    # add trajectory
    size_list = [6] * trajectory.shape[0]
    size_list[0] = 10
    size_list[-1] = 10

    color_list = ['rgba(4, 174, 4, 0.5)'] * trajectory.shape[0]
    color_list[0] = 'rgba(12, 5, 235, 1)'
    color_list[-1] = 'rgba(235, 5, 5, 1)'

    position_count = {}
    text_list = []
    for i in range(trajectory.shape[0]):
        if str(trajectory[i]) in position_count:
            position_count[str(trajectory[i])] += 1
        else:
            position_count[str(trajectory[i])] = 0
        text_list.append('        ' * position_count[str(trajectory[i])] + f'{i}')
    if full_name:
        text_list[0] = f'{name} Start Point: 0'
        text_list[-1] = f'{name} End Point: {trajectory.shape[0] - 1}'
    return size_list, color_list, text_list
    

def visualize_trajectory(trajectory, floor_plan_filename, width_meter, height_meter, title=None, mode='lines + markers + text', show=False):
    fig = go.Figure()

    size_list, color_list, text_list = get_lists_of_one_trajectory(trajectory)

    fig.add_trace(
        go.Scattergl(
            x=trajectory[:, 0],
            y=trajectory[:, 1],
            mode=mode,
            marker=dict(size=size_list, color=color_list),
            line=dict(shape='linear', color='rgb(100, 10, 100)', width=2, dash='dot'),
            text=text_list,
            textposition="top center",
            name='trajectory',
        ))

    # add floor plan
    floor_plan = Image.open(floor_plan_filename)
    fig.update_layout(images=[
        go.layout.Image(
            source=floor_plan,
            xref="x",
            yref="y",
            x=0,
            y=height_meter,
            sizex=width_meter,
            sizey=height_meter,
            sizing="contain",
            opacity=1,
            layer="below",
        )
    ])

    # configure
    fig.update_xaxes(autorange=False, range=[0, width_meter])
    fig.update_yaxes(autorange=False, range=[0, height_meter], scaleanchor="x", scaleratio=1)
    fig.update_layout(
        title=go.layout.Title(
            text=title or "No title.",
            xref="paper",
            x=0,
        ),
        autosize=True,
        width=900,
        height=200 + 900 * height_meter / width_meter,
        template="plotly_white",
    )

    if show:
        fig.show()

    return fig


def visualize_trajectories(trajectories, floor_plan_filename, width_meter, height_meter, title=None, mode='lines + markers + text', show=False):
    fig = go.Figure()

    for c,trajectory in enumerate(trajectories):
        size_list, color_list, text_list = get_lists_of_one_trajectory(trajectory,False)
        fig.add_trace(
            go.Scattergl(
                x=trajectory[:, 0],
                y=trajectory[:, 1],
                mode=mode,
                marker=dict(size=size_list, color=color_list),
                line=dict(shape='linear', color='rgb(100, 10, 100)', width=2, dash='dot'),
                text=text_list,
                textposition="top center",
                name=None#'trajectory',
            ))

    # add floor plan
    floor_plan = Image.open(floor_plan_filename)
    fig.update_layout(images=[
        go.layout.Image(
            source=floor_plan,
            xref="x",
            yref="y",
            x=0,
            y=height_meter,
            sizex=width_meter,
            sizey=height_meter,
            sizing="contain",
            opacity=1,
            layer="below",
        )
    ])

    # configure
    fig.update_xaxes(autorange=False, range=[0, width_meter])
    fig.update_yaxes(autorange=False, range=[0, height_meter], scaleanchor="x", scaleratio=1)
    fig.update_layout(
        title=go.layout.Title(
            text=title or "No title.",
            xref="paper",
            x=0,
        ),
        autosize=True,
        width=900,
        height=200 + 900 * height_meter / width_meter,
        template="plotly_white",
    )

    if show:
        fig.show()

    return fig


def visualize_heatmap(position, value, floor_plan_filename, width_meter, height_meter, colorbar_title="colorbar", title=None, show=False):
    fig = go.Figure()

    # add heat map
    fig.add_trace(
        go.Scatter(x=position[:, 0],
                   y=position[:, 1],
                   mode='markers',
                   marker=dict(size=7,
                               color=value,
                               colorbar=dict(title=colorbar_title),
                               colorscale="Rainbow"),
                   text=value,
                   name=title))

    # add floor plan
    floor_plan = Image.open(floor_plan_filename)
    fig.update_layout(images=[
        go.layout.Image(
            source=floor_plan,
            xref="x",
            yref="y",
            x=0,
            y=height_meter,
            sizex=width_meter,
            sizey=height_meter,
            sizing="contain",
            opacity=1,
            layer="below",
        )
    ])

    # configure
    fig.update_xaxes(autorange=False, range=[0, width_meter])
    fig.update_yaxes(autorange=False, range=[0, height_meter], scaleanchor="x", scaleratio=1)
    fig.update_layout(
        title=go.layout.Title(
            text=title or "No title.",
            xref="paper",
            x=0,
        ),
        autosize=True,
        width=900,
        height=200 + 900 * height_meter / width_meter,
        template="plotly_white",
    )

    if show:
        fig.show()

    return fig

In [ ]:
def visualize_train_trajectory(path):
    _id, floor = path.split("/")[:2]
    
    train_floor_data = read_data_file(os.path.join("../input/indoor-location-navigation/train/", path))
    with open(f"../input/indoor-location-navigation/metadata/{_id}/{floor}/floor_info.json") as f:
        train_floor_info = json.load(f)
    #print(train_floor_data.waypoint)
    return visualize_trajectory(
        train_floor_data.waypoint[:, 1:3], 
        f"../input/indoor-location-navigation/metadata/{_id}/{floor}/floor_image.png",
        train_floor_info["map_info"]["width"], 
        train_floor_info["map_info"]["height"],
    )

def visualize_train_trajectories(paths):
    # must be from the same building and the same floor
    fps = set([os.path.dirname(path) for path in paths])
    assert len(fps) == 1
    
    _id, floor = paths[0].split("/")[-3:-1]
    print(_id, floor)
    
    with open(f"{PATH}/metadata/{_id}/{floor}/floor_info.json") as f:
        train_floor_info = json.load(f)
    
    trajectories = []
    for path in paths:
        train_floor_data = read_data_file(path)
        trajectories.append(train_floor_data.waypoint[:, 1:3])

        
    return visualize_trajectories(
        trajectories, 
        f"{PATH}/metadata/{_id}/{floor}/floor_image.png",
        train_floor_info["map_info"]["width"], 
        train_floor_info["map_info"]["height"],
        title = f"{_id} {floor}", show=True
    )

In [ ]:
floorplans = sorted(glob.glob(f"{PATH}/train/*/*"))
print("Number of floor plans:", len(floorplans))
floorplans[:5]

In [ ]:
paths = {fp:glob.glob(f"{fp}/*.txt") for fp in floorplans}

### Show one building

Since there are many floorplans, let's just show one building in this notebook, you can also show them all if you want.

In [ ]:
building_0 = floorplans[0].split('/')[-2]
print('Building', building_0)
for fp in floorplans:
    if fp.split('/')[-2]!=building_0:
        break
    paths = glob.glob(f"{fp}/*.txt")
    visualize_train_trajectories(paths)

### Show them all!!
Uncomment and run the following code

In [ ]:
"""
for fp in floorplans:
    paths = glob.glob(f"{fp}/*.txt")
    visualize_train_trajectories(paths)
"""